# Непосредственно работа с mfd_ria_news.csv

In [19]:
import pandas as pd
from nltk.tokenize import TreebankWordTokenizer
from collections import Counter, defaultdict
from nltk.util import ngrams
import numpy as np

In [6]:
df = pd.read_csv('./project/mfd_ria_news.csv', names=['news_date', 'news_text','url'])

In [70]:
df.head()

,news_date,news_text,url,column_with_tokens,three_grams,two_grams
0,28.05.2021,"МОСКВА, 28 мая - РИА Новости/Прайм. Мировые це...",https://mfd.ru/news/view/?id=2425835,"[28, мая, Мировые, цены, на, нефть, растут, в,...","[(28, мая, Мировые), (мая, Мировые, цены), (Ми...","[(28, мая), (мая, Мировые), (Мировые, цены), (..."
1,28.05.2021,"МОСКВА, 28 мая - РИА Новости/Прайм. Средневзве...",https://mfd.ru/news/view/?id=2425675,"[28, мая, Средневзвешенный, курс, доллара, к, ...","[(28, мая, Средневзвешенный), (мая, Средневзве...","[(28, мая), (мая, Средневзвешенный), (Средневз..."
2,27.05.2021,"МОСКВА, 27 мая - РИА Новости/Прайм, Сергей Аст...",https://mfd.ru/news/view/?id=2425520,"[27, мая, Сергей, Астахов., Российский, рынок,...","[(27, мая, Сергей), (мая, Сергей, Астахов.), (...","[(27, мая), (мая, Сергей), (Сергей, Астахов.),..."
3,27.05.2021,"МОСКВА, 27 мая - РИА Новости/Прайм. Мировые це...",https://mfd.ru/news/view/?id=2425435,"[27, мая, Мировые, цены, на, нефть, в, четверг...","[(27, мая, Мировые), (мая, Мировые, цены), (Ми...","[(27, мая), (мая, Мировые), (Мировые, цены), (..."
4,26.05.2021,"МОСКВА, 26 мая - РИА Новости/Прайм, Дмитрий Ма...",https://mfd.ru/news/view/?id=2425330,"[26, мая, Дмитрий, Майоров., Рубль, вечером, м...","[(26, мая, Дмитрий), (мая, Дмитрий, Майоров.),...","[(26, мая), (мая, Дмитрий), (Дмитрий, Майоров...."


In [8]:
df.news_text[1][:-169]

'МОСКВА, 28 мая - РИА Новости/Прайм. Средневзвешенный курс доллара к рублю расчетами "завтра" на единой торговой сессии (ЕТС) Московской биржи, по которому устанавливается официальный курс ЦБ, на 11.30 мск пятницы вырос на 12,90 копейки, составив 73,5870 рубля, свидетельствуют данные биржи\u200b\u200b\u200b.   В четверг средневзвешенный курс доллара составлял 73,4580 рубля за доллар.'

In [5]:
def get_tokens(sentence):
    sentence = sentence[:-169]
    tokenizer = TreebankWordTokenizer()
    sentence = tokenizer.tokenize(sentence)
    new_sentence = [x for x in sentence if x and x not in '- \t\n.,;:!?(—)/©' \
                    and x not in ['ria.ru', 'РИА', 'Новости', 'МОСКВА', 'Новости/Прайм', '``', "''", 'Новости/Прайм.']]
    return new_sentence

In [9]:
df['column_with_tokens'] = df.news_text.apply(get_tokens)

In [16]:
def get_counter_for_column_with_tokens(column_with_tokens=df['column_with_tokens']):
    spisok = []
    for article in column_with_tokens:
        spisok = spisok + article
    return Counter(spisok)

In [ ]:
count = get_counter_for_column_with_tokens(column_with_tokens=df['column_with_tokens'])

In [29]:
count.most_common(50000)[49980:49999]

NameError: name 'count' is not defined

In [33]:
len(sorted(count)) # - число уникальных слов/токенов

67838

### Думаю, что слова которые встречаются менее 10 раз (а это отсекается на уровне уже 8 тыс.слов), имеет смысл удалить. Информативности ноль, если ты встретился в 4 тыс. статьях лишь 10 раз. Пока думаю так, чтобы уменьшить словарь с 67 тыс. слов до 8 тыс.

In [24]:
df['three_grams'] = df['column_with_tokens'].apply(lambda x: list(ngrams(x, 3)))

In [51]:
df['two_grams'] = df['column_with_tokens'].apply(lambda x: list(ngrams(x, 2)))

In [59]:
c = []
for item in df['three_grams']:
    c = c + item

In [60]:
count_three_grams = Counter(c)

In [69]:
count_three_grams.most_common(10)

[(('доллара', 'за', 'баррель'), 1734),
 (('на', 'нефть', 'марки'), 1282),
 (('следует', 'из', 'данных'), 1263),
 (('доллара', 'расчетами', 'завтра'), 1258),
 (('рубля', 'курс', 'евро'), 1074),
 (('долларов', 'за', 'баррель'), 1000),
 (('цены', 'на', 'нефть'), 907),
 (('из', 'данных', 'Московской'), 882),
 (('рубля', 'следует', 'из'), 840),
 (('торгов.По', 'состоянию', 'на'), 826)]

# С помощью регулярных выражений можно делать простые выделители смыслов по условию if ... else...

In [1]:
import re

In [2]:
r = '(hi|hello|hey)[]*([a-z]*)'

In [3]:
from itertools import permutations

In [4]:
list(permutations('Good morning Rosa!'.split(), 3))

[('Good', 'morning', 'Rosa!'),
 ('Good', 'Rosa!', 'morning'),
 ('morning', 'Good', 'Rosa!'),
 ('morning', 'Rosa!', 'Good'),
 ('Rosa!', 'Good', 'morning'),
 ('Rosa!', 'morning', 'Good')]

# Токенизация

In [1]:
sentence = 'Thomas Jefferson began building Monticello at the age of 26.'

In [2]:
sentence.split()

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

In [3]:
import numpy as np

In [4]:
token_sequense = str.split(sentence)

In [5]:
vocab = sorted(set(token_sequense))

In [6]:
vocab

['26.',
 'Jefferson',
 'Monticello',
 'Thomas',
 'age',
 'at',
 'began',
 'building',
 'of',
 'the']

In [7]:
', '.join(vocab)

'26., Jefferson, Monticello, Thomas, age, at, began, building, of, the'

In [8]:
num_tokens = len(token_sequense)

In [9]:
vocab_size = len(vocab)

In [10]:
onehot_vectors = np.zeros((num_tokens, vocab_size), int)

In [11]:
onehot_vectors

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [12]:
for i, word in enumerate(token_sequense):
    onehot_vectors[i, vocab.index(word)]=1

In [13]:
onehot_vectors

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [14]:
' '.join(vocab)

'26. Jefferson Monticello Thomas age at began building of the'

In [15]:
import pandas as pd

In [16]:
pd.DataFrame(onehot_vectors, columns=vocab)

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0,0,0,0


In [17]:
df = pd.DataFrame(onehot_vectors, columns=vocab)

In [18]:
df[df == 0] = ''

In [19]:
df

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,,,,1,,,,,,
1,,1,,,,,,,,
2,,,,,,,1,,,
3,,,,,,,,1,,
4,,,1,,,,,,,
5,,,,,,1,,,,
6,,,,,,,,,,1
7,,,,,1,,,,,
8,,,,,,,,,1,
9,1,,,,,,,,,


In [20]:
sentence_bow = {}
for token in sentence.split():
    sentence_bow[token] = 1
sorted(sentence_bow.items())

[('26.', 1),
 ('Jefferson', 1),
 ('Monticello', 1),
 ('Thomas', 1),
 ('age', 1),
 ('at', 1),
 ('began', 1),
 ('building', 1),
 ('of', 1),
 ('the', 1)]

In [21]:
import pandas as pd
df = pd.DataFrame(pd.Series(dict([(token, 1) for token in sentence.split()])), columns=['sent']).T

In [22]:
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.
sent,1,1,1,1,1,1,1,1,1,1


In [34]:
sentences = 'Thomas Jefferson began building Monticello at the\
age of 26.\n'

In [36]:
sentences+='Construction was done mostly by local masons and carpenters.\n'

In [37]:
sentences+='He moved in to the South Pavilion in 1770.\n'

In [38]:
sentences+="Turning Monticello into neoclassical masterpiece was Jefferson's obsession"

In [39]:
corpus = {}

In [40]:
for i, sent in enumerate(sentences.split('\n')):
    corpus['sent{}'.format(i)] = dict((tok, 1) for tok in sent.split())

In [41]:
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T

In [42]:
df

,Thomas,Jefferson,began,building,Monticello,at,theage,of,26.,Construction,...,the,South,Pavilion,1770.,Turning,into,neoclassical,masterpiece,Jefferson's,obsession
sent0,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
sent1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
sent2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,0
sent3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1


In [44]:
df[df.columns[:15]]

,Thomas,Jefferson,began,building,Monticello,at,theage,of,26.,Construction,was,done,mostly,by,local
sent0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0
sent1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1
sent2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
sent3,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


# Скалярное произведение - один из способов нахождения сходства между предложениями

In [45]:
df = df.T
df.sent0.dot(df.sent1)

0

In [46]:
df.sent0.dot(df.sent2)

0

In [47]:
df.sent0.dot(df.sent3)

1

### *Корпус - это словарь, показывающий сколько раз встречается каждое слово. Слово - ключ, количество раз - это значение*

Находим общее слово в sent1 и sent3:

In [49]:
[(k, v) for (k, v) in (df.sent0 & df.sent3).items() if v]

[('Monticello', 1)]

Это ваша первая **модель векторного пространства (VSM) документов** (предложений) на естественном языке

In [50]:
import re

In [51]:
sentence = 'Thomas Jefferson began building Monticello at the age of 26.'

In [55]:
tokens = re.split(r'[-\s.,;]+', sentence)

In [58]:
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '']

In [59]:
pattern = re.compile(r"([-\s.,;!?])+")
tokens = pattern.split(sentence)
tokens[-10:] # только последние 10 токенов
['the', ' ', 'age', ' ', 'of', ' ', '26', '.', '']

['the', ' ', 'age', ' ', 'of', ' ', '26', '.', '']

In [60]:
sentence = 'Thomas Jefferson began building Monticello at the age of 26.'

In [61]:
tokens = pattern.split(sentence)

Избавляемся от пробелов, точек и др. в токенах:

In [63]:
[x for x in tokens if x and x not in '- \t\n.,;!?']

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

Таким образом, встроенный re прекрасно подходит для токенизации и фильтрации

In [64]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+|$[0-9.]+|\S+')
tokenizer.tokenize(sentence)

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '.']

Один из лучших токенизаторов:

In [65]:
from nltk.tokenize import TreebankWordTokenizer
sentence = """Monticello wasn't designated as UNESCO World Heritage Site until 1987."""
tokenizer = TreebankWordTokenizer()
tokenizer.tokenize(sentence)

['Monticello',
 'was',
 "n't",
 'designated',
 'as',
 'UNESCO',
 'World',
 'Heritage',
 'Site',
 'until',
 '1987',
 '.']

In [67]:
s = """ Россия ввела запрет на экспорт гречки
10:21 05.06.2021 (обновлено: 11:59 05.06.2021)
6858
Гречневая крупа, продающаяся на развес  - РИА Новости, 1920, 05.06.2021
© РИА Новости / Константин Михальчевский
Перейти в фотобанк
Читать ria.ru в
МОСКВА, 5 июн — РИА Новости. С 5 июня в России вступил в силу временный запрет на вывоз гречки; он будет действовать до 31 августа. Правительство приняло соответствующее постановление в конце апреля."""

In [68]:
s

' Россия ввела запрет на экспорт гречки\n10:21 05.06.2021 (обновлено: 11:59 05.06.2021)\n6858\nГречневая крупа, продающаяся на развес  - РИА Новости, 1920, 05.06.2021\n© РИА Новости / Константин Михальчевский\nПерейти в фотобанк\nЧитать ria.ru в\nМОСКВА, 5 июн — РИА Новости. С 5 июня в России вступил в силу временный запрет на вывоз гречки; он будет действовать до 31 августа. Правительство приняло соответствующее постановление в конце апреля.'

In [70]:
ts = tokenizer.tokenize(s)

In [84]:
[x for x in ts if x and x not in '- \t\n.,;:!?(—)/©' and x not in ['ria.ru', 'РИА', 'Новости', 'МОСКВА']]

['Россия',
 'ввела',
 'запрет',
 'на',
 'экспорт',
 'гречки',
 '10:21',
 '05.06.2021',
 'обновлено',
 '11:59',
 '05.06.2021',
 '6858',
 'Гречневая',
 'крупа',
 'продающаяся',
 'на',
 'развес',
 '1920',
 '05.06.2021',
 'Константин',
 'Михальчевский',
 'Перейти',
 'в',
 'фотобанк',
 'Читать',
 'в',
 '5',
 'июн',
 'Новости.',
 'С',
 '5',
 'июня',
 'в',
 'России',
 'вступил',
 'в',
 'силу',
 'временный',
 'запрет',
 'на',
 'вывоз',
 'гречки',
 'он',
 'будет',
 'действовать',
 'до',
 '31',
 'августа.',
 'Правительство',
 'приняло',
 'соответствующее',
 'постановление',
 'в',
 'конце',
 'апреля']

### Токенизация неформального текста из социальных сетей, таких как Twitter и Facebook

In [86]:
from nltk.tokenize.casual import casual_tokenize
message = """RT @TJMonticello Best day everrrrrrr at Monticello. Awesommmmmmeeeeeeee day :*)"""
casual_tokenize(message)

['RT',
 '@TJMonticello',
 'Best',
 'day',
 'everrrrrrr',
 'at',
 'Monticello',
 '.',
 'Awesommmmmmeeeeeeee',
 'day',
 ':*)']

In [87]:
casual_tokenize(message, reduce_len=True, strip_handles=True)

['RT',
 'Best',
 'day',
 'everrr',
 'at',
 'Monticello',
 '.',
 'Awesommmeee',
 'day',
 ':*)']

# Расширяем словарь n-граммами

In [88]:
sentence = """Thomas Jefferson began building Monticello at the age of 26."""
pattern = re.compile(r"([-\s.,;!?])+")
tokens = pattern.split(sentence)
tokens = [x for x in tokens if x and x not in '- \t\n.,;!?']
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

In [89]:
from nltk.util import ngrams
list(ngrams(tokens, 2))

[('Thomas', 'Jefferson'),
 ('Jefferson', 'began'),
 ('began', 'building'),
 ('building', 'Monticello'),
 ('Monticello', 'at'),
 ('at', 'the'),
 ('the', 'age'),
 ('age', 'of'),
 ('of', '26')]

In [90]:
list(ngrams(tokens, 3))

[('Thomas', 'Jefferson', 'began'),
 ('Jefferson', 'began', 'building'),
 ('began', 'building', 'Monticello'),
 ('building', 'Monticello', 'at'),
 ('Monticello', 'at', 'the'),
 ('at', 'the', 'age'),
 ('the', 'age', 'of'),
 ('age', 'of', '26')]

In [91]:
two_grams = list(ngrams(tokens, 2))
[" ".join(x) for x in two_grams]

['Thomas Jefferson',
 'Jefferson began',
 'began building',
 'building Monticello',
 'Monticello at',
 'at the',
 'the age',
 'age of',
 'of 26']

# Стоп-слова. Их не удаляем чтобы не терять информативность

In [53]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/board/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [54]:
stop_words = nltk.corpus.stopwords.words('russian')

In [55]:
len(stop_words)

151

# Выравнивание регистра - нормализация

In [71]:
tokens = ['House', 'Visitor', 'Center']

In [72]:
normalized_tokens = [x.lower() for x in tokens]

In [73]:
print(normalized_tokens)

['house', 'visitor', 'center']


# Стемминг - работа с окончаниями

In [1]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
' '.join([stemmer.stem(w).strip("'") for w in "dish washer's washed dishes".split()])

'dish washer wash dish'

Русскоязычный стеммер (скачал файл с гитхаба):

In [2]:
from russian_stemmer import Stemmer

In [3]:
stemmer = Stemmer()

In [6]:
stemmer.stem('окончательный')

'окончательн'

In [7]:
stemmer.stem('сессии')

'сесс'

In [24]:
stemmer.stem('усилили')

'усил'

Или можно использовать этот русскоязычный стеммер в NLTK:

In [14]:
from nltk.stem.snowball import SnowballStemmer 

stemmer = SnowballStemmer("russian") 
stemmer.stem("Василий")
'Васил'

'Васил'

In [15]:
stemmer.stem('перешедший')

'перешедш'

In [16]:
stemmer.stem('реализацию')

'реализац'

In [17]:
stemmer.stem('знающего')

'знающ'

In [25]:
stemmer.stem('усилили')

'усил'

# Лемматизация

In [27]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/board/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [28]:
from nltk.stem import WordNetLemmatizer

In [29]:
lemmatizer = WordNetLemmatizer()

In [30]:
lemmatizer.lemmatize('better')

'better'

In [31]:
lemmatizer.lemmatize('better', pos='a')

'good'

### *http://www.solarix.ru/* - полезные вещи для русскоязычного NLP

Установил **русский лемматизатор**, код ниже

In [46]:
# сначала скачал с гитхаба https://github.com/Koziev/rulemma , файл rulemma-master.zip. Потом запустил
#!pip3 install /home/board/Загрузки/rulemma-master.zip
# pip3 install /home/board/Загрузки/rutokenizer-master.zip   - ссылка там же, на рулемме Козиева. Остальное там же
# pip3 install /home/board/Загрузки/rusyllab-master.zip
# pip3 install /home/board/Загрузки/ruword2tags-master.zip
# pip3 install /home/board/Загрузки/rupostagger-master.zip
# !pip3 install python-crfsuite

С проектами выше вылезает ошибка - файл не является database. Залез в исходный код, могу потратить день-два, попробовать понять ошибку обращения к базе данных. Но похоже нижеследующий проект - круче:
https://habr.com/ru/post/516098/
Или даже DeepPavlov...  требует мощный процессор, но похоже Наташа выполняет как надо все

Начну с основного проекта Наташи - **pip3 install natasha**. Запускаю пример использования, и вижу разбиение на части речи!...

In [ ]:
from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    
    Doc
)

segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

text = 'Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры...'
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)

sent = doc.sents[0]
# sent.morph.print()

Вот эти этапы надо выполнить обязательно для лемматизации:

In [16]:
doc2 = Doc(text)
doc2.segment(segmenter)
doc2.tag_morph(morph_tagger)
doc2.parse_syntax(syntax_parser)

Попробую решить задачу лемматизации:

In [17]:
from natasha import MorphVocab
morph_vocab = MorphVocab()
for token in doc2.tokens:
    token.lemmatize(morph_vocab)

In [18]:
{_.text: _.lemma for _ in doc.tokens}

{'Посол': 'посол',
 'Израиля': 'израиль',
 'на': 'на',
 'Украине': 'украина',
 'Йоэль': 'йоэль',
 'Лион': 'лион',
 'признался': 'признаться',
 ',': ',',
 'что': 'что',
 'пришел': 'прийти',
 'в': 'в',
 'шок': 'шок',
 'узнав': 'узнать',
 'о': 'о',
 'решении': 'решение',
 'властей': 'власть',
 'Львовской': 'львовский',
 'области': 'область',
 'объявить': 'объявить',
 '2019': '2019',
 'год': 'год',
 'годом': 'год',
 'лидера': 'лидер',
 'запрещенной': 'запретить',
 'России': 'россия',
 'Организации': 'организация',
 'украинских': 'украинский',
 'националистов': 'националист',
 '(': '(',
 'ОУН': 'оун',
 ')': ')',
 'Степана': 'степан',
 'Бандеры': 'бандера',
 '...': '...'}

### !!! КРУТО !!! ТО ЧТО НУЖНО ДЛЯ СОЗДАНИЯ СЖАТОГО СЛОВАРЯ!!! Сразу решается и вопросы стемминга и регистров слов...

 # ЧТО НАШЛОСЬ, ПАРНИ... https://github.com/natasha/corus
 Links to publicly available Russian corpora + code for loading and parsing. 20+ datasets, 350Gb+ of text.
 Ссылки на датасеты новостей и т.п.

In [1]:
path = '/home/board/lenta-ru-news.csv.gz'

In [2]:
from corus import load_lenta
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [ ]:
records = load_lenta(path)
c = 0
for record in records:
    c = c +1
    if c > 10000:
        break
    if c%300 == 0:
        print(record.text)
        print(record.url)

Новостей от ленты.ру - дофига короче. 500 мб, десятки-сотней новостей ежедневно. Можно в цикле ставит условие - если в новости есть упоминание курса доллара допустим, тогда только эта новость отбираеся. Это разовая работа по очистке датасета

# Тональность

VADER — анализатор тональности на основе правил. Я так понял только для английского языка.

Вы можете найти более детальную информацию о процессе установки на GitHub по адресу
**github.com/cjhutto/vaderSentiment**

Если вы еще не установили nlpia, инструкции по установке можно найти по адресу **github.com/
totalgood/nlpia**

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
sa = SentimentIntensityAnalyzer()

In [10]:
len(sa.lexicon)

7506

In [7]:
[(tok, score) for tok, score in sa.lexicon.items() if ' ' in tok]

[("( '}{' )", 1.6),
 ("can't stand", -2.0),
 ('fed up', -1.8),
 ('screwed up', -1.5)]

In [9]:
sa.polarity_scores(text='Python is very readable and it is great for NLP')

{'neg': 0.0, 'neu': 0.687, 'pos': 0.313, 'compound': 0.6249}

In [14]:
corpus = ['Absolutely perfect! Love it! :-) :-) :-)', 'Horrible! Completely useless', 'It was ok. Some good \
            and some bad things']

In [15]:
for doc in corpus:
    scores = sa.polarity_scores(doc)
    print('{:+}: {}'.format(scores['compound'], doc))

+0.9428: Absolutely perfect! Love it! :-) :-) :-)
-0.7836: Horrible! Completely useless
-0.1531: It was ok. Some good             and some bad things


# Наивный байесовский классификатор для измерения тональности

In [10]:
import pandas as pd
file_ = '/home/board/.local/lib/python3.8/site-packages/nlpia/data/hutto_ICWSM_2014/movieReviewSnippets_GroundTruth.csv.gz'
movies = pd.read_csv(file_, index_col='id')

In [13]:
movies.head()

,sentiment,text
id,,
1,2.266667,The Rock is destined to be the 21st Century's ...
2,3.533333,The gorgeously elaborate continuation of ''The...
3,-0.600000,Effective but too tepid biopic
4,1.466667,If you sometimes like to go to the movies to h...
5,1.733333,"Emerges as something rare, an issue movie that..."


In [14]:
movies.describe().round(2)

,sentiment
count,10605.00
mean,0.00
std,1.92
min,-3.88
25%,-1.77
50%,-0.08
75%,1.83
max,3.94


In [15]:
import pandas as pd  # noqa
pd.set_option('display.width', 75)
from nltk.tokenize import casual_tokenize  # noqa
bags_of_words = []
from collections import Counter

In [16]:
for text in movies.text:
    bags_of_words.append(Counter(casual_tokenize(text)))

In [18]:
len(bags_of_words)

10605

In [ ]:
df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)
df_bows.shape
# (10605, 20756)
df_bows.head()